IMPORTS

In [1]:
import numpy as np
import cv2
import os

SHOW IMAGE FUNCTION

In [2]:
# Function from lab2
def show_image(image, window_name='image', timeout=0):
    cv2.imshow(window_name, cv2.resize(image, None, fx=0.6, fy=0.6))
    cv2.waitKey(timeout)
    cv2.destroyAllWindows()

FUNCTION TO READ IMAGES FROM FILE

In [3]:
# Read images from dataset
def read_images():
    img = []
    folder_path = 'Task1/'
    
    for i in range(1, 26):
        filename = '0' + str(i) if i < 10 else str(i)
        img_path = os.path.join(folder_path, filename + '.jpg')
        img.append(cv2.imread(img_path))
    
    return img

FUNCTION TO FIND THE SCORE AREAS

In order to find all circles of interest (the ones with points), I used findContours method from cv2, after that multiple contours were detected so I analysed the image and removed the ones with width and height smaller than 180/190, the one representing the frame of the whole image and also two extra which were found on the edge of the dartboard

In [4]:
# This function will return all circles of interest
def find_circles(image):

    # Convert the image to grayscale
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Filter image and find contours.
    ret, thresh = cv2.threshold(image_gray, 150, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(image=thresh, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
                                      
    # Make a copy because the original image will be changed
    image_copy = image.copy()
    
    # In this variable we will try to store the 9 segments with points
    best_contours = []
    for c in contours:
        x,y,w,h = cv2.boundingRect(c)
        # Very small segments were detected so I removed those
        if w > 180 and h > 190:
            # I observed that the frame of the whole image is also included, so by this condition I removed it
            if w < 2400 and h < 3200:
                # print(w, h)
                best_contours.append(c)
  
    # I observed that the first two contours are located outside the board so I decided to remove those.
    del best_contours[:2]
    # # Test to see if the regions were found
    # for i in range(9):
    #     cv2.drawContours(image=image_copy, contours=best_contours[i], contourIdx=-1, color=(0, 0, 70 + i*20), thickness=13, lineType=cv2.LINE_AA)
    # show_image(image_copy, 'test', 0)
    return best_contours


In [5]:
# Apply a threshold so that we can detect the pixels of interest, we isolate the colored pixels
# seeing that the image is predominantly black and white with darts colored.
def detectDarts(image):
    hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower = np.array([65, 50, 65])
    upper = np.array([125, 255, 255])
    mask = cv2.inRange(hsv_img, lower, upper)

    # Detecting and drawing contours
    # First we find the contours in the filtered image
    copy = mask.copy()
    kernel = np.ones((50, 50), np.uint8)
    img_dilation = cv2.dilate(mask, kernel, iterations=1)
    edges = cv2.Canny(img_dilation, 50, 150)
    contours, hierarchy = cv2.findContours(img_dilation,
                                           cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cv2.drawContours(image=img_dilation, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)

    # We search for the figures with height and width bigger than a value chosen through testing
    darts_num = 0
    darts = []
    for c in contours:
        x,y,w,h = cv2.boundingRect(c)
        if w > 90 and h > 90:
            darts.append(c)
            darts_num += 1

    return darts, darts_num

FUNCTION TO DETECT THE POINT REPRESENTING THE TIP OF THE DART

In order to find this point I computed first the centre of the poligon representing the flag, and after analysing all the pictures in the train dataset I observed that most of the darts tips are located around 450 units to the left and 10 units to the bottom relative to the centre of the flag, so I used this heuristic method in order to determine the tip.

In [6]:
def detectTipPosition(darts, img):
    # Inspiration for the blob center computation:
    # https://learnopencv.com/find-center-of-blob-centroid-using-opencv-cpp-python/
    tips_list = []
    for dart in darts:
        dart_center = cv2.moments(dart)
        # Computing the centre for each dart using moments method from cv2.
        cX = int(dart_center["m10"] / dart_center["m00"])
        cY = int(dart_center["m01"] / dart_center["m00"])
        
        # Estimate the tip.
        cA = int(cX - 450)
        cB = int(cY + 10)
        tips_list.append((cA, cB))

        # # Code for drawing a marker on tip (testing purposes)
        # color = (0, 255, 0)
        # markerType = cv2.MARKER_CROSS
        # markerSize = 50
        # thickness = 5
        # cv2.drawMarker(img, (cA, cB), color, markerType, markerSize, thickness)
    return tips_list
    

FUNCTION TO ESTIMATE THE SCORE FOR A TIP OF THE DART

Start with the smaller circle and check if it contains the point representing the tip of the dart if not, it will go to the next smaller circle and check again

In [7]:
# We expect the countours to be sorted in ascending order by area.
# Start with the smaller circle and check if it contains the tip of the dart
# if not, it will go to the next bigger circle.
def estimateScore(countours, tips_list):
    scores_list = []
    
    for arrow in tips_list:
        for i in range(9):
            if cv2.pointPolygonTest(countours[i], arrow, False) >= 0:
                scores_list.append(9 - i)
                break
    return scores_list

FUNCTION TO WRITE THE PREDICTION IN FILE

In [8]:
def writeInFile(darts_num, scores_list, i):
    file_identifier = i + 1
    filename = ('0' + str(file_identifier) if file_identifier < 10 else str(file_identifier))
    path = 'Ilicea_Anca_512/Task1/' + filename + '_predicted.txt'

    print(path)
    f = open(path, 'w')
    f.write(str(darts_num) + '\n')

    for score in scores_list:
        f.write(str(score) + '\n')

MAIN FUNCTION, IT WILL RUN ALL THE FUNCTIONS AND GENERATE THE SOLUTION

In [9]:
def main():
    # Get the template so we can calculate the elipsis
    template = cv2.imread('auxiliary_images/template_task1.jpg')
    # Get all the train images
    img_array = read_images()

    # Iterate through all images
    for i, img in enumerate(img_array):
        # Detect the number of darts
        darts, darts_num = detectDarts(img)
        # Get the tip point for each dart
        tips_list = detectTipPosition(darts, img)

        # Get the areas of interest
        best_contours = find_circles(template)
        # I noticed that the find_circles function I made always returned
        # the contours in order from big to small, so I decided to use that
        # and reverse it for our next step, score estimation
        best_contours.reverse()
        scores_list = estimateScore(best_contours, tips_list)
        
        writeInFile(darts_num, scores_list, i)
    
main()

Ilicea_Anca_512/Task1/01_predicted.txt
Ilicea_Anca_512/Task1/02_predicted.txt
Ilicea_Anca_512/Task1/03_predicted.txt
Ilicea_Anca_512/Task1/04_predicted.txt
Ilicea_Anca_512/Task1/05_predicted.txt
Ilicea_Anca_512/Task1/06_predicted.txt
Ilicea_Anca_512/Task1/07_predicted.txt
Ilicea_Anca_512/Task1/08_predicted.txt
Ilicea_Anca_512/Task1/09_predicted.txt
Ilicea_Anca_512/Task1/10_predicted.txt
Ilicea_Anca_512/Task1/11_predicted.txt
Ilicea_Anca_512/Task1/12_predicted.txt
Ilicea_Anca_512/Task1/13_predicted.txt
Ilicea_Anca_512/Task1/14_predicted.txt
Ilicea_Anca_512/Task1/15_predicted.txt
Ilicea_Anca_512/Task1/16_predicted.txt
Ilicea_Anca_512/Task1/17_predicted.txt
Ilicea_Anca_512/Task1/18_predicted.txt
Ilicea_Anca_512/Task1/19_predicted.txt
Ilicea_Anca_512/Task1/20_predicted.txt
Ilicea_Anca_512/Task1/21_predicted.txt
Ilicea_Anca_512/Task1/22_predicted.txt
Ilicea_Anca_512/Task1/23_predicted.txt
Ilicea_Anca_512/Task1/24_predicted.txt
Ilicea_Anca_512/Task1/25_predicted.txt
